In [18]:
#instalar pip install feature-engine

import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.linear_model import Lasso
from sklearn.feature_selection import SelectFromModel
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler, Binarizer

from sklearn.pipeline import Pipeline

from feature_engine.imputation import(
    AddMissingIndicator
    ,MeanMedianImputer
    ,CategoricalImputer
)

from feature_engine.encoding import (
    RareLabelEncoder
    ,OrdinalEncoder
)

from feature_engine.transformation import LogTransformer

from feature_engine.selection import DropFeatures
from feature_engine.wrappers import SklearnTransformerWrapper


import joblib

In [9]:
!conda install -c conda-forge feature_engine

^C


In [16]:
!pip install feature_engine

  Using cached feature_engine-1.1.2-py2.py3-none-any.whl (180 kB)


In [20]:
import my_preprocessors as mypp #nosotros

In [21]:
data=pd.read_csv("train.csv")
data.head()

Id  MSSubClass MSZoning  LotFrontage  LotArea Street Alley LotShape  \
0   1          60       RL         65.0     8450   Pave   NaN      Reg   
1   2          20       RL         80.0     9600   Pave   NaN      Reg   
2   3          60       RL         68.0    11250   Pave   NaN      IR1   
3   4          70       RL         60.0     9550   Pave   NaN      IR1   
4   5          60       RL         84.0    14260   Pave   NaN      IR1   

  LandContour Utilities  ... PoolArea PoolQC Fence MiscFeature MiscVal MoSold  \
0         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
1         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      5   
2         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      9   
3         Lvl    AllPub  ...        0    NaN   NaN         NaN       0      2   
4         Lvl    AllPub  ...        0    NaN   NaN         NaN       0     12   

  YrSold  SaleType  SaleCondition  SalePrice  
0   2008        WD         Normal     208500  
1   2007        WD         Normal     181500  
2   2008        WD         Normal     223500  
3   2006        WD        Abnorml     140000  
4   2008        WD         Normal     250000  

[5 rows x 81 columns]

In [23]:
#Cast de variables MSSubClass
data['MSSubClass'] = data['MSSubClass'].astype('O')

In [25]:
X_train, X_test, y_train, y_test = train_test_split(
        data.drop(['Id', 'SalePrice'], axis=1),
    data['SalePrice'],
    test_size=0.1,
    random_state=2022
)

X_train.shape, X_test.shape

((1314, 79), (146, 79))

In [26]:
### Transformación al target
y_train = np.log(y_train)
y_test = np.log(y_test)

### Configuración de machine Learning pipeline

In [27]:
#Variables categoricas con NA

CATEGORICAL_VARS_WITH_NA_FREQUENT = ['BsmtQual', 'BsmtExposure',
                                     'BsmtFinType1', 'GarageFinish']

#Variable categórica con NA pero indicador de missing
CATEGORICAL_VARS_WITH_NA_MISSING = ['FireplaceQu']

#Variables numericas con NA
NUMERICAL_VARS_WITH_NA = ['LotFrontage']

#vARIABLES de temporalidad
TEMPORAL_VARS = ['YearRemodAdd']
REF_VAR = "YrSold"

#Variables que vamos a tirar
DROP_FEATURES = ["YrSold"]

#Variables para transformación logaritmica
NUMERICALS_LOG_VARS = ["LotFrontage", "1stFlrSF", "GrLivArea"]


#Variables para binarización paor sesgo fuerte
BINARIZE_VARS = ['ScreenPorch']

#Variables para hacer mapeo categórico por codificación ordinal
QUAL_VARS = ['ExterQual', 'BsmtQual',
             'HeatingQC', 'KitchenQual', 'FireplaceQu']


#Variables para hacer mapeo categórico por codificación sin ordinal
EXPOSURE_VARS = ['BsmtExposure']

FINISH_VARS = ['BsmtFinType1']

GARAGE_VARS = ['GarageFinish']

FENCE_VARS = ['Fence']


###
CATEGORICAL_VARS = ['MSSubClass',  'MSZoning',  'LotShape',  'LandContour',
                    'LotConfig', 'Neighborhood', 'RoofStyle', 'Exterior1st',
                    'Foundation', 'CentralAir', 'Functional', 'PavedDrive',
                    'SaleCondition']

QUAL_MAPPINGS = {'Po': 1, 'Fa': 2, 'TA': 3,
                 'Gd': 4, 'Ex': 5, 'Missing': 0, 'NA': 0}

EXPOSURE_MAPPINGS = {'No': 1, 'Mn': 2, 'Av': 3, 'Gd': 4}

FINISH_MAPPINGS = {'Missing': 0, 'NA': 0, 'Unf': 1,
                   'LwQ': 2, 'Rec': 3, 'BLQ': 4, 'ALQ': 5, 'GLQ': 6}

GARAGE_MAPPINGS = {'Missing': 0, 'NA': 0, 'Unf': 1, 'RFn': 2, 'Fin': 3}

#variables seleccionadas según analisis de lasso
FEATURES = [
    'MSSubClass',
    'MSZoning',
    'LotFrontage',
    'LotShape',
    'LandContour',
    'LotConfig',
    'Neighborhood',
    'OverallQual',
    'OverallCond',
    'YearRemodAdd',
    'RoofStyle',
    'Exterior1st',
    'ExterQual',
    'Foundation',
    'BsmtQual',
    'BsmtExposure',
    'BsmtFinType1',
    'HeatingQC',
    'CentralAir',
    '1stFlrSF',
    '2ndFlrSF',
    'GrLivArea',
    'BsmtFullBath',
    'HalfBath',
    'KitchenQual',
    'TotRmsAbvGrd',
    'Functional',
    'Fireplaces',
    'FireplaceQu',
    'GarageFinish',
    'GarageCars',
    'GarageArea',
    'PavedDrive',
    'WoodDeckSF',
    'ScreenPorch',
    'SaleCondition',
    # this one is only to calculate temporal variable:
    "YrSold",
]


### Machine learning pipeline

In [ ]:
housePrice_pipeline = Pipeline([
    
    #====== IMPUTACIONES ==========#
    
    #IMPUTACION DE VARIABLES CATEGORICAS
    #INICIAMOS CON NOMBRE
    #segundo función que se usa para ejecutar esa operacion, tienen que tener un metodo fit y transform
    ('missing_imputation', CategoricalImputer(imputation_method='missing'
                                             ,variables= CATEGORICAL_VARS_WITH_NA_MISSING)
    )
    ,
    #2. IMPUTACION DE VARIABLES 
    ('frequent_imputation', CategoricalImputer(imputation_method='frequent'
                                             ,variables= CATEGORICAL_VARS_WITH_NA_FREQUENT)
    )
    ,
    #2. IMPUTACION DE VARIABLES 
    ('frequent_imputation', CategoricalImputer(imputation_method='frequent'
                                             ,variables= CATEGORICAL_VARS_WITH_NA_FREQUENT)
    )
    ,
    #3. indicamos faltante en variables numericas para imputar 
    ('missing_indicator', AddMissingIndicator(variables=NUMERICAL_VARS_WITH_NA))
    ,
    #4. imputacion de mediana para variables categoricas
    ('mean_imputation', MeanMedianImputer(
        imputation_method='mean', variables =NUMERICAL_VARS_WITH_NA)
    )
    ,
    
    #========= VARIABLES TEMPORALES ===========#
    
    #5. Tratamiento de variables temporales
    ('eslapsed_time', mypp.TemporalVariableTransformer(
    variables = TEMPORAL_VARS, reference_variable = REF_VAR)
    
    )
    ,
    
    #6. Droop de variables
    ('drop_featres', DropFeatures(features_to_drop=DROP_FEATURES)
    )
    ,
    
    #============= TRANSFORMACION DE VARIABLES NUMERICAS ===============#
    
    #7. Transformacion logaritmica
    ('log', LogTransformer(variables = NUMERICALS_LOG_VARS)
    )
    ,
    
    #8. bINARIZACION DE VARIABLES CON SESGO FUERTE
    ('binarizer', SklearnTransformerWrapper(transformer = Binarizer(threshold =0)
                                           , variables = BINARIZE_VARS)
    )
    ,
    
    
    
])